## Homework

> Note: sometimes your answer doesn't match one of the options exactly.
> That's fine.
> Select the option that's closest to your solution.

### Dataset

In this homework, we will use the lead scoring dataset Bank Marketing dataset. Download it from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv).

Or you can do it with `wget`:

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
```

In this dataset our desired target for classification task will be `converted` variable - has the client signed up to the platform or not.





In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv -O data.csv

--2025-10-15 01:35:01--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]  78.98K  --.-KB/s    in 0.006s  

2025-10-15 01:35:01 (13.4 MB/s) - ‘data.csv’ saved [80876/80876]



In [2]:
import pandas as pd
df = pd.read_csv('data.csv')
df.shape

(1462, 9)

In [3]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


### Data preparation

* Check if the missing values are presented in the features.
* If there are missing values:
    * For categorical features, replace them with 'NA'
    * For numerical features, replace with with 0.0 





In [4]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
numerical = df.select_dtypes(include=['int64', 'float64'])
numerical = numerical.drop(columns=['converted'])
numerical.columns

Index(['number_of_courses_viewed', 'annual_income', 'interaction_count',
       'lead_score'],
      dtype='object')

In [6]:
categorical = df.select_dtypes(include=['object'])
categorical.columns

Index(['lead_source', 'industry', 'employment_status', 'location'], dtype='object')

In [7]:
df[numerical.columns] = df[numerical.columns].fillna(0.0)
df[categorical.columns] = df[categorical.columns].fillna('NA')

In [8]:
for c in categorical.columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [9]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

### Question 1

What is the most frequent observation (mode) for the column `industry`?

- `NA`
- `technology`
- `healthcare`
- `retail`

In [10]:
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
na               134
Name: count, dtype: int64

In [11]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

Answer 1 = retail

### Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset.
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- `interaction_count` and `lead_score`
- `number_of_courses_viewed` and `lead_score`
- `number_of_courses_viewed` and `interaction_count`
- `annual_income` and `interaction_count`

Only consider the pairs above when answering this question.





In [12]:
cormatrix = df[numerical.columns].corr().round(3)
cormatrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000,0.010,-0.024,-0.005
annual_income,0.010,1.000,0.027,0.016
interaction_count,-0.024,0.027,1.000,0.010
lead_score,-0.005,0.016,0.010,1.000


interaction_count and lead_score = 0.010

number_of_courses_viewed and lead_score = -0.005

number_of_courses_viewed and interaction_count = -0.024

annual_income and interaction_count = 0.027

Answer: annual_income and interaction_count

### Split the data

- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
- Make sure that the target value `converted` is not in your dataframe.

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [15]:
len(df_train), len(df_val), len(df_test)


(876, 293, 293)

In [16]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop=True)

In [17]:
y_train = df_train['converted'].values
y_val = df_val['converted'].values
y_test = df_test['converted'].values
y_full_train = df_full_train['converted'].values
del df_train['converted']
del df_val['converted']
del df_test['converted']
del df_full_train['converted']

In [18]:
df_train.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77
2,paid_ads,technology,3,81973.0,employed,north_america,2,0.59
3,na,technology,1,74956.0,employed,europe,3,0.34
4,organic_search,retail,3,59335.0,student,australia,1,0.98


### Question 3

- Calculate the mutual information score between `converted` and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?

- `industry`
- `location`
- `lead_source`
- `employment_status`



In [19]:
from sklearn.metrics import mutual_info_score

In [20]:
for col in ('industry','location','lead_source','employment_status'):
    print(col, round(mutual_info_score(df_train[col], y_train),2))

industry 0.01
location 0.0
lead_source 0.04
employment_status 0.01


Answer: lead_source

### Question 4

- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
  - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
  - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.64
- 0.74
- 0.84
- 0.94


In [21]:
from sklearn.feature_extraction import DictVectorizer

In [22]:
dv = DictVectorizer(sparse=False)
train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
X_train.shape

(876, 31)

In [23]:
val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)
X_val.shape

(293, 31)

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42, C=1.0)
model.fit(X_train, y_train)
model.intercept_[0], model.coef_[0]

(np.float64(-0.06914728027832559),
 array([-1.77843866e-05,  3.39095225e-02, -1.47154423e-02,  2.66248432e-03,
         1.15238518e-02, -1.02527697e-01,  4.93604222e-02, -2.01258344e-02,
        -1.34214865e-02, -3.00232200e-03, -2.48510995e-02, -9.25991830e-03,
        -3.17957304e-02, -1.60513114e-02,  3.11339155e-01,  5.12012528e-02,
        -1.20346284e-02,  2.01511698e-02, -1.16021521e-02, -1.15251880e-01,
         7.95303436e-02, -2.99401329e-02, -1.14296944e-02, -1.12457415e-02,
        -5.59987025e-03,  8.26402635e-03,  5.58598769e-03,  3.95843295e-03,
        -3.33967159e-02, -2.52837052e-02,  4.53752887e-01]))

In [26]:
y_pred = model.predict_proba(X_val)[:, 1]
convert_predict = (y_pred >= 0.5).astype(int)
round((convert_predict == y_val).mean(),2)

np.float64(0.7)

In [51]:
(convert_predict == y_val).mean()

np.float64(0.6996587030716723)

Answer: Accuracy = 0.70

### Question 5

- Let's find the least useful feature using the _feature elimination_ technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

- `'industry'`
- `'employment_status'`
- `'lead_score'`

> **Note**: The difference doesn't have to be positive.


In [34]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, y_full_train)

In [42]:

categorical_names = categorical.columns.tolist()
categorical_names

['lead_source', 'industry', 'employment_status', 'location']

In [ ]:

subset_names = categorical.columns.tolist() + numerical.columns.tolist()
subset_names


['lead_source',
 'employment_status',
 'location',
 'number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [43]:
mi = df_full_train[categorical_names].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

lead_source          0.025665
employment_status    0.013258
industry             0.011685
location             0.002253
dtype: float64

In [49]:
def TrainModelWithout(colname):
    subset_names = categorical.columns.tolist() + numerical.columns.tolist()
    subset_names.remove(colname)
    print (subset_names)
    train_dict = df_train[subset_names].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42, C=1.0)
    model.fit(X_train, y_train)
    val_dict = df_val[subset_names].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    y_pred = model.predict_proba(X_val)[:, 1]
    convert_predict = (y_pred >= 0.5).astype(int)
    return (convert_predict == y_val).mean()

In [52]:
def TrainModel(series, Cval):
    train_dict = series.to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42, C=Cval)
    model.fit(X_train, y_train)
    val_dict = df_val[subset_names].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    y_pred = model.predict_proba(X_val)[:, 1]
    convert_predict = (y_pred >= 0.5).astype(int)
    return (convert_predict == y_val).mean()

In [50]:
for col in ('industry','employment_status','lead_score'):
    print(col, TrainModelWithout(col))


['lead_source', 'employment_status', 'location', 'number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
industry 0.6996587030716723
['lead_source', 'industry', 'location', 'number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
employment_status 0.6962457337883959
['lead_source', 'industry', 'employment_status', 'location', 'number_of_courses_viewed', 'annual_income', 'interaction_count']
lead_score 0.7064846416382252


Answer: Least difference = without industry

### Question 6

- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter `C`: `[0.01, 0.1, 1, 10, 100]`.
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these `C` leads to the best accuracy on the validation set?

- 0.01
- 0.1
- 1
- 10
- 100

> **Note**: If there are multiple options, select the smallest `C`.


In [53]:
for c in (0.01, 0.1, 1, 10,100):
    print(c, TrainModel(df_train, c))

0.01 0.689419795221843
0.1 0.6996587030716723
1 0.6996587030716723
10 0.6996587030716723
100 0.6996587030716723


Answer C = 0.1


## Submit the results

- Submit your results here: https://courses.datatalks.club/ml-zoomcamp-2025/homework/hw03
- If your answer doesn't match options exactly, select the closest one